In [1]:
%%capture
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score

In [2]:
QPATH = "Quantlet/4-qode2desc"

In [3]:
import sys

IN_COLAB = "google.colab" in sys.modules

import os

if IN_COLAB:
    os.chdir(
        f"/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}"
    )
else:
    %load_ext lab_black

In [13]:
import pandas as pd

In [4]:
from tqdm import tqdm

tqdm.pandas()

import torch
import torch, gc

os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import nltk

nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
import importlib
import analysis_modules

importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/usr/net/zinovyee.hub/IRTG/MLSC/Encode-the-Qode/Quantlet/4-qode2desc/analysis_modules.py'>

In [6]:
def create_name(analysis_config):
    name = analysis_config["model_name"]
    if "checkpoint" in name:
        name = name.split("/")[-1]
    mode = analysis_config["MODE"]
    date = analysis_config["DATE"]
    if analysis_config["val_data_name"].startswith("val"):
        sample = "val"
    else:
        sample = "test"
    return f"{name}_{mode}_{sample}_{date}"

In [7]:
analysis_config = {
    "DATE": "20231022",
    "MODE": "domain",
    "model_name": "CodeT5",
    "encoder_max_length": 512,
    "decoder_max_length": 75,
    "random_state": 42,
    "learning_rate": 5e-5,
    "epochs": 4,
    "train_batch": 16,
    "eval_batch": 4,
    "warmup_steps": 500,
    "weight_decay": 0.1,
    "logging_stes": 100,
    "save_total_lim": 1,
    "save_strategy": "steps",
    "label_smooting": 0.1,
    "predict_generate": True,
    "load_best_model_at_end": False,
    "evaluation_strategy": "epoch",
}
if analysis_config["MODE"] == "domain":
    analysis_config[
        "train_data_path"
    ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/no_context/"
else:
    analysis_config[
        "train_data_path"
    ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/{analysis_config['MODE']}/"

analysis_config["train_data_name"] = (
    f"full_train_dataset_{analysis_config['DATE']}_sample0.json",
)
if analysis_config["MODE"] == "domain":
    analysis_config[
        "val_data_path"
    ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/no_context/"
else:
    analysis_config[
        "val_data_path"
    ] = f"../../data/preprocessed/Quantlet/{analysis_config['DATE']}/{analysis_config['MODE']}/"
analysis_config[
    "val_data_name"
] = f"test_dataset_{analysis_config['DATE']}_sample0.json"
analysis_config["analysis_name"] = create_name(analysis_config)
print(analysis_config["analysis_name"])

if analysis_config["MODE"] == "domain":
    if analysis_config["model_name"] == "CodeT5":
        analysis_config[
            "model_name"
        ] = "../../data/pretrained/analysis_report_CodeT5-test-12-300-4-2023-09-26-v2/results/checkpoint-88488"
    if analysis_config["model_name"] == "CodeTrans":
        analysis_config[
            "model_name"
        ] = "../../data/pretrained/CodeTrans/results/checkpoint-12290"

CodeT5_domain_test_20231022


In [8]:
gc.collect()
torch.cuda.empty_cache()

In [10]:
trainer = analysis_modules.scs_analyze(**analysis_config)

CodeT5_domain_test_20231022
cuda
cuda


Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-7c5e6e4837b55da6/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Found cached dataset json (/home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-9aac8b7225e27cff/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96)


  0%|          | 0/1 [00:00<?, ?it/s]

Loading cached processed dataset at /home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-7c5e6e4837b55da6/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-4ace09ff0c81cbaf.arrow
Loading cached processed dataset at /home/RDC/zinovyee.hub/.cache/huggingface/datasets/json/default-9aac8b7225e27cff/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96/cache-6b7d1c64f0adc5a9.arrow
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      9.529        0.123        0.026        0.107            0.11   

   eval_bleu  eval_gen_len  
0      0.001        13.967  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,6.083000,4.770382,0.266400,0.085100,0.226400,0.232500,17.689000,0.021500,0.313300,0.462900,5945,12844
2,4.306300,4.241068,0.294700,0.110100,0.249300,0.256600,17.733700,0.035800,0.350000,0.487900,6266,12844
3,3.508500,4.039348,0.306500,0.136700,0.264300,0.269300,17.930900,0.049800,0.363000,0.496700,6379,12844
4,3.131300,4.016418,0.310300,0.140800,0.267700,0.274000,17.987800,0.052900,0.365000,0.498100,6397,12844


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.016         0.31        0.141        0.268           0.274   

   eval_bleu  eval_gen_len  
0      0.053        17.988  
__________
Original: Algorithm that identify the S&P500 firm tickers from news text. The algorithm only identify those tickers in the three most commonly seen cases. Detailed description refer to the paper


Summary before Tuning: Dense Text Cleaning and Parsing from Nltk Import concurrent


Summary after Tuning: This Quantlet is dedicated to classification of the Solidity Price of the Solidity based Turkish website. We provide three methods to visualise the effect of changing the effect of the effect on the effect of the keyword. For each of the methods, we select one of the names of the publications and use the names of the publications. For


__________



__________
Original: Inspect the derived variables from the News Network, Plots and Stats


Summary before Tuning: Analysis of Netw

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [46]:
def parse_logs(trainer):
    log_history = trainer.state.log_history
    train_log = pd.DataFrame(columns=log_history[0].keys())
    eval_log = pd.DataFrame(columns=log_history[1].keys())
    for log in log_history:
        if "loss" in log:
            train_log = pd.concat(
                [train_log, pd.DataFrame.from_dict(log, orient="index").T], axis=0
            )
        elif "eval_loss" in log:
            eval_log = pd.concat(
                [eval_log, pd.DataFrame.from_dict(log, orient="index").T], axis=0
            )

    logs = train_log.merge(
        eval_log,
        how="inner",
        left_on=["epoch", "step"],
        right_on=["epoch", "step"],
    )
    return logs[
        [
            "epoch",
            "loss",
            "step",
            "eval_loss",
            "eval_rouge1",
            "eval_rouge2",
            "eval_rougeL",
            "eval_rougeLsum",
            "eval_gen_len",
            "eval_bleu",
            "eval_brevity_penalty",
            "eval_length_ratio",
            "eval_translation_length",
            "eval_reference_length",
        ]
    ]